- Reference: https://zhuanlan.zhihu.com/p/72279816

In [2]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

os.makedirs("images", exist_ok=True)

parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=28, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument("--sample_interval", type=int, default=400, help="interval betwen image samples")
opt = parser.parse_args([])
print(opt)

img_shape = (opt.channels, opt.img_size, opt.img_size)
cuda = True if torch.cuda.is_available() else False

class Generator(nn.Module):
    """ 生成器 """
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img


class Discriminator(nn.Module):
    """ 评价器 """
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity


# Loss function 损失函数
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Configure data loader
os.makedirs("./data/mnist", exist_ok=True)
dataset_transform = transforms.Compose([
        transforms.Resize(opt.img_size), 
        transforms.ToTensor(), 
        transforms.Normalize([0.5], [0.5])
    ])
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        root="./data/mnist",
        train=True,
        download=True,
        transform=dataset_transform
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

Namespace(b1=0.5, b2=0.999, batch_size=64, channels=1, img_size=28, latent_dim=100, lr=0.0002, n_cpu=8, n_epochs=200, sample_interval=400)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/mnist\MNIST\raw\train-images-idx3-ubyte.gz to ./data/mnist\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/mnist\MNIST\raw\train-labels-idx1-ubyte.gz to ./data/mnist\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data/mnist\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/mnist\MNIST\raw



# Training 

In [3]:
# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

[Epoch 0/200] [Batch 0/938] [D loss: 0.716419] [G loss: 0.717128]
[Epoch 0/200] [Batch 1/938] [D loss: 0.631977] [G loss: 0.713688]
[Epoch 0/200] [Batch 2/938] [D loss: 0.568123] [G loss: 0.710730]
[Epoch 0/200] [Batch 3/938] [D loss: 0.511626] [G loss: 0.707380]
[Epoch 0/200] [Batch 4/938] [D loss: 0.464901] [G loss: 0.703454]
[Epoch 0/200] [Batch 5/938] [D loss: 0.435687] [G loss: 0.698248]
[Epoch 0/200] [Batch 6/938] [D loss: 0.405865] [G loss: 0.692409]
[Epoch 0/200] [Batch 7/938] [D loss: 0.391210] [G loss: 0.685243]
[Epoch 0/200] [Batch 8/938] [D loss: 0.384805] [G loss: 0.677435]
[Epoch 0/200] [Batch 9/938] [D loss: 0.381243] [G loss: 0.667142]
[Epoch 0/200] [Batch 10/938] [D loss: 0.383061] [G loss: 0.657069]
[Epoch 0/200] [Batch 11/938] [D loss: 0.386062] [G loss: 0.643040]
[Epoch 0/200] [Batch 12/938] [D loss: 0.389887] [G loss: 0.633202]
[Epoch 0/200] [Batch 13/938] [D loss: 0.397020] [G loss: 0.621136]
[Epoch 0/200] [Batch 14/938] [D loss: 0.406118] [G loss: 0.607593]
[Epoc

[Epoch 0/200] [Batch 123/938] [D loss: 0.497171] [G loss: 1.010942]
[Epoch 0/200] [Batch 124/938] [D loss: 0.473015] [G loss: 0.910508]
[Epoch 0/200] [Batch 125/938] [D loss: 0.443934] [G loss: 0.971043]
[Epoch 0/200] [Batch 126/938] [D loss: 0.434817] [G loss: 1.229479]
[Epoch 0/200] [Batch 127/938] [D loss: 0.417437] [G loss: 1.108490]
[Epoch 0/200] [Batch 128/938] [D loss: 0.397511] [G loss: 1.302374]
[Epoch 0/200] [Batch 129/938] [D loss: 0.400669] [G loss: 1.334145]
[Epoch 0/200] [Batch 130/938] [D loss: 0.414767] [G loss: 1.268309]
[Epoch 0/200] [Batch 131/938] [D loss: 0.407560] [G loss: 1.338611]
[Epoch 0/200] [Batch 132/938] [D loss: 0.441698] [G loss: 1.136751]
[Epoch 0/200] [Batch 133/938] [D loss: 0.433257] [G loss: 1.261529]
[Epoch 0/200] [Batch 134/938] [D loss: 0.448395] [G loss: 1.165974]
[Epoch 0/200] [Batch 135/938] [D loss: 0.477403] [G loss: 1.005475]
[Epoch 0/200] [Batch 136/938] [D loss: 0.471110] [G loss: 1.173533]
[Epoch 0/200] [Batch 137/938] [D loss: 0.479692]

[Epoch 0/200] [Batch 246/938] [D loss: 0.486639] [G loss: 0.960084]
[Epoch 0/200] [Batch 247/938] [D loss: 0.509760] [G loss: 0.817861]
[Epoch 0/200] [Batch 248/938] [D loss: 0.540298] [G loss: 0.900693]
[Epoch 0/200] [Batch 249/938] [D loss: 0.541864] [G loss: 0.679381]
[Epoch 0/200] [Batch 250/938] [D loss: 0.539696] [G loss: 0.965412]
[Epoch 0/200] [Batch 251/938] [D loss: 0.574011] [G loss: 0.554554]
[Epoch 0/200] [Batch 252/938] [D loss: 0.584190] [G loss: 1.195400]
[Epoch 0/200] [Batch 253/938] [D loss: 0.668233] [G loss: 0.383070]
[Epoch 0/200] [Batch 254/938] [D loss: 0.636745] [G loss: 1.133637]
[Epoch 0/200] [Batch 255/938] [D loss: 0.610448] [G loss: 0.467892]
[Epoch 0/200] [Batch 256/938] [D loss: 0.518976] [G loss: 0.889439]
[Epoch 0/200] [Batch 257/938] [D loss: 0.492606] [G loss: 0.885150]
[Epoch 0/200] [Batch 258/938] [D loss: 0.536844] [G loss: 0.731602]
[Epoch 0/200] [Batch 259/938] [D loss: 0.514823] [G loss: 0.883195]
[Epoch 0/200] [Batch 260/938] [D loss: 0.529249]

[Epoch 0/200] [Batch 366/938] [D loss: 0.500132] [G loss: 1.215683]
[Epoch 0/200] [Batch 367/938] [D loss: 0.475151] [G loss: 0.694681]
[Epoch 0/200] [Batch 368/938] [D loss: 0.492956] [G loss: 1.273906]
[Epoch 0/200] [Batch 369/938] [D loss: 0.524839] [G loss: 0.634043]
[Epoch 0/200] [Batch 370/938] [D loss: 0.489498] [G loss: 1.332278]
[Epoch 0/200] [Batch 371/938] [D loss: 0.502182] [G loss: 0.633646]
[Epoch 0/200] [Batch 372/938] [D loss: 0.509434] [G loss: 1.379285]
[Epoch 0/200] [Batch 373/938] [D loss: 0.534430] [G loss: 0.553288]
[Epoch 0/200] [Batch 374/938] [D loss: 0.553652] [G loss: 1.503631]
[Epoch 0/200] [Batch 375/938] [D loss: 0.602740] [G loss: 0.442218]
[Epoch 0/200] [Batch 376/938] [D loss: 0.518729] [G loss: 1.426592]
[Epoch 0/200] [Batch 377/938] [D loss: 0.495255] [G loss: 0.697825]
[Epoch 0/200] [Batch 378/938] [D loss: 0.481756] [G loss: 0.964313]
[Epoch 0/200] [Batch 379/938] [D loss: 0.510991] [G loss: 0.940641]
[Epoch 0/200] [Batch 380/938] [D loss: 0.543210]

[Epoch 0/200] [Batch 487/938] [D loss: 0.467003] [G loss: 1.211588]
[Epoch 0/200] [Batch 488/938] [D loss: 0.442548] [G loss: 1.004475]
[Epoch 0/200] [Batch 489/938] [D loss: 0.507311] [G loss: 0.786238]
[Epoch 0/200] [Batch 490/938] [D loss: 0.473967] [G loss: 0.946801]
[Epoch 0/200] [Batch 491/938] [D loss: 0.478750] [G loss: 0.997857]
[Epoch 0/200] [Batch 492/938] [D loss: 0.504665] [G loss: 0.881273]
[Epoch 0/200] [Batch 493/938] [D loss: 0.454045] [G loss: 0.917441]
[Epoch 0/200] [Batch 494/938] [D loss: 0.467239] [G loss: 1.071159]
[Epoch 0/200] [Batch 495/938] [D loss: 0.452900] [G loss: 0.778911]
[Epoch 0/200] [Batch 496/938] [D loss: 0.451149] [G loss: 1.518095]
[Epoch 0/200] [Batch 497/938] [D loss: 0.521965] [G loss: 0.570966]
[Epoch 0/200] [Batch 498/938] [D loss: 0.493109] [G loss: 1.630663]
[Epoch 0/200] [Batch 499/938] [D loss: 0.561512] [G loss: 0.506395]
[Epoch 0/200] [Batch 500/938] [D loss: 0.623408] [G loss: 1.589995]
[Epoch 0/200] [Batch 501/938] [D loss: 0.663099]

...

[Epoch 26/200] [Batch 135/938] [D loss: 0.516262] [G loss: 0.777906]
[Epoch 26/200] [Batch 136/938] [D loss: 0.640683] [G loss: 8.107463]
[Epoch 26/200] [Batch 137/938] [D loss: 0.218357] [G loss: 4.627118]
[Epoch 26/200] [Batch 138/938] [D loss: 0.603862] [G loss: 0.596340]
[Epoch 26/200] [Batch 139/938] [D loss: 0.503434] [G loss: 8.478943]
[Epoch 26/200] [Batch 140/938] [D loss: 0.433432] [G loss: 7.619077]
[Epoch 26/200] [Batch 141/938] [D loss: 0.486477] [G loss: 2.119628]
[Epoch 26/200] [Batch 142/938] [D loss: 0.091621] [G loss: 5.911926]
[Epoch 26/200] [Batch 143/938] [D loss: 0.441096] [G loss: 6.103601]
[Epoch 26/200] [Batch 144/938] [D loss: 0.449667] [G loss: 3.245966]
[Epoch 26/200] [Batch 145/938] [D loss: 0.067586] [G loss: 5.547162]
[Epoch 26/200] [Batch 146/938] [D loss: 0.117330] [G loss: 5.324652]
[Epoch 26/200] [Batch 147/938] [D loss: 0.223898] [G loss: 3.852807]
[Epoch 26/200] [Batch 148/938] [D loss: 0.191286] [G loss: 3.057451]
[Epoch 26/200] [Batch 149/938] [D 

[Epoch 26/200] [Batch 254/938] [D loss: 0.151632] [G loss: 2.275082]
[Epoch 26/200] [Batch 255/938] [D loss: 0.164970] [G loss: 2.773352]
[Epoch 26/200] [Batch 256/938] [D loss: 0.113022] [G loss: 2.243656]
[Epoch 26/200] [Batch 257/938] [D loss: 0.162978] [G loss: 2.635128]
[Epoch 26/200] [Batch 258/938] [D loss: 0.113410] [G loss: 2.369911]
[Epoch 26/200] [Batch 259/938] [D loss: 0.146095] [G loss: 2.983733]
[Epoch 26/200] [Batch 260/938] [D loss: 0.165502] [G loss: 2.140687]
[Epoch 26/200] [Batch 261/938] [D loss: 0.096283] [G loss: 2.570285]
[Epoch 26/200] [Batch 262/938] [D loss: 0.162225] [G loss: 2.786916]
[Epoch 26/200] [Batch 263/938] [D loss: 0.143206] [G loss: 1.655895]
[Epoch 26/200] [Batch 264/938] [D loss: 0.294249] [G loss: 5.300457]
[Epoch 26/200] [Batch 265/938] [D loss: 0.184212] [G loss: 2.592312]
[Epoch 26/200] [Batch 266/938] [D loss: 0.434064] [G loss: 0.741526]
[Epoch 26/200] [Batch 267/938] [D loss: 0.801895] [G loss: 8.765045]
[Epoch 26/200] [Batch 268/938] [D 

[Epoch 26/200] [Batch 372/938] [D loss: 0.101652] [G loss: 3.900014]
[Epoch 26/200] [Batch 373/938] [D loss: 0.157414] [G loss: 3.956441]
[Epoch 26/200] [Batch 374/938] [D loss: 0.216931] [G loss: 2.605428]
[Epoch 26/200] [Batch 375/938] [D loss: 0.107995] [G loss: 3.225997]
[Epoch 26/200] [Batch 376/938] [D loss: 0.180984] [G loss: 3.896329]
[Epoch 26/200] [Batch 377/938] [D loss: 0.231043] [G loss: 1.551245]
[Epoch 26/200] [Batch 378/938] [D loss: 0.429323] [G loss: 5.520198]
[Epoch 26/200] [Batch 379/938] [D loss: 0.145569] [G loss: 2.421166]
[Epoch 26/200] [Batch 380/938] [D loss: 0.225967] [G loss: 1.314677]
[Epoch 26/200] [Batch 381/938] [D loss: 0.384237] [G loss: 6.015443]
[Epoch 26/200] [Batch 382/938] [D loss: 0.135650] [G loss: 3.784045]
[Epoch 26/200] [Batch 383/938] [D loss: 0.326742] [G loss: 1.360326]
[Epoch 26/200] [Batch 384/938] [D loss: 0.208658] [G loss: 6.743252]
[Epoch 26/200] [Batch 385/938] [D loss: 0.127681] [G loss: 5.702940]
[Epoch 26/200] [Batch 386/938] [D 

[Epoch 26/200] [Batch 492/938] [D loss: 0.233786] [G loss: 1.823551]
[Epoch 26/200] [Batch 493/938] [D loss: 0.099482] [G loss: 3.581858]
[Epoch 26/200] [Batch 494/938] [D loss: 0.384003] [G loss: 3.221278]
[Epoch 26/200] [Batch 495/938] [D loss: 0.372115] [G loss: 0.976580]
[Epoch 26/200] [Batch 496/938] [D loss: 0.305698] [G loss: 3.841242]
[Epoch 26/200] [Batch 497/938] [D loss: 0.214343] [G loss: 3.327402]
[Epoch 26/200] [Batch 498/938] [D loss: 0.209056] [G loss: 1.717281]
[Epoch 26/200] [Batch 499/938] [D loss: 0.200494] [G loss: 2.727575]
[Epoch 26/200] [Batch 500/938] [D loss: 0.216666] [G loss: 2.384951]
[Epoch 26/200] [Batch 501/938] [D loss: 0.152994] [G loss: 1.995303]
[Epoch 26/200] [Batch 502/938] [D loss: 0.151701] [G loss: 2.861815]
[Epoch 26/200] [Batch 503/938] [D loss: 0.123742] [G loss: 2.613563]
[Epoch 26/200] [Batch 504/938] [D loss: 0.153089] [G loss: 2.304989]
[Epoch 26/200] [Batch 505/938] [D loss: 0.184629] [G loss: 2.448919]
[Epoch 26/200] [Batch 506/938] [D 

KeyboardInterrupt: 